# CatBoost Model

In [2]:
# This version number is appended to each generated file as a suffix
version = 120

In [3]:
# Importing the required libraries
import pandas as pd
import numpy as np

## Loading Data

In [4]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v20.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v20.csv")
submission_set = pd.read_csv("./data/final_submission_set.csv")

## Predictive Model Learning

In [4]:
# Defining the categorical features names
cat_names = ['callsign',
            'adep', 
            'ades', 
            'aircraft_type', 
            'wtc', 
            'airline',
            'offblock_hour',
            'offblock_minute', 
            'offblock_day_of_week',
            'offblock_weekday_name',
            'offblock_month',
            'offblock_week_of_year', 
            'offblock_season', 
            'arrival_hour',
            'arrival_minute',
            'arrival_season',
            'arrival_weekday_name',
            'is_offblock_weekend',
            'is_offblock_rush_hour',
            'flight_duration_category',                       
            'adep_region', 
            'ades_region', 
            'same_country_flight',
            'same_region_flight',                        
            'flight_direction',
            'is_intercontinental',
            'Manufacturer',
            'Model_FAA',
            'Physical_Class_Engine',
            'FAA_Weight',
            'adep_geo_cluster',
            'ades_geo_cluster']

In [5]:
# Since we have both the challenge and submission sets locally, we will concatenate them to do data imputation (i.e. removing NaNs)
dataset = pd.concat([challenge_set_updated, submission_set_updated], axis=0)

In [6]:
# Changing display options so we can see all columns from the dataset
pd.set_option('display.max_rows', None)

# Verifying the percentage of missing values for each column in the dataset
print(dataset.isnull().mean().sort_values(ascending=False))

pd.reset_option('display.max_rows')

callsign                            0.000000
adep                                0.000000
ades                                0.000000
aircraft_type                       0.000000
wtc                                 0.000000
airline                             0.000000
taxiout_time                        0.000000
flown_distance                      0.000000
track_variation_ARR_100             0.040820
track_variation_DEP_40              0.031541
track_variation_ENR                 0.000670
average_vertical_rate_ARR_100       0.040809
average_vertical_rate_DEP_40        0.031444
average_vertical_rate_ENR           0.000660
average_airspeed_ARR_100            0.040820
average_airspeed_DEP_40             0.031541
average_airspeed_ENR                0.000670
groundspeed_ARR_100                 0.040820
groundspeed_DEP_40                  0.031541
groundspeed_ENR                     0.000670
wind_distance_ARR_100               0.000000
wind_distance_DEP_40                0.000000
wind_dista

In [7]:
# Dropping columns with too many NaNs
threshold = 0.4
df = dataset.dropna(thresh=int((1 - threshold) * len(dataset)), axis=1)

In [8]:
# Imputation of NaNs
columns_with_nan = dataset.isna().any()
for col in dataset.columns[columns_with_nan]:
    dataset.loc[:, col] = dataset.fillna(dataset[col].median())

In [9]:
pd.set_option('display.max_rows', None)

# Verifying if we indeed removed all NaNs
print(dataset.isnull().mean().sort_values(ascending=False))

pd.reset_option('display.max_rows')

callsign                            0.0
adep                                0.0
ades                                0.0
aircraft_type                       0.0
wtc                                 0.0
airline                             0.0
taxiout_time                        0.0
flown_distance                      0.0
track_variation_ARR_100             0.0
track_variation_DEP_40              0.0
track_variation_ENR                 0.0
average_vertical_rate_ARR_100       0.0
average_vertical_rate_DEP_40        0.0
average_vertical_rate_ENR           0.0
average_airspeed_ARR_100            0.0
average_airspeed_DEP_40             0.0
average_airspeed_ENR                0.0
groundspeed_ARR_100                 0.0
groundspeed_DEP_40                  0.0
groundspeed_ENR                     0.0
wind_distance_ARR_100               0.0
wind_distance_DEP_40                0.0
wind_distance_ENR                   0.0
average_temperature_ARR_100         0.0
average_temperature_DEP_40          0.0


In [10]:
# After imputation, we recover the challenge set portion
df = dataset.iloc[0:challenge_set_updated.shape[0], :]

In [11]:
X = df.drop('tow', axis=1)
y = df.tow

In [12]:
# Features eliminated by select_features() (see catboost_select.ipynb)
eliminated_features = ['groundspeed_airspeed_ratio_ENR', 'temperature_9', 'wind_distance_flown_distance_ENR', 'average_humidity_DEP_40', 'vertical_rate_bins_ARR', 
        'groundspeed_flown_distance_ARR', 'arrival_quarter', 'offblock_year', 'arrival_year', 'offblock_to_arrival_day_diff', 'altitude_9', 'tas_1', 
        'is_arrival_weekend', 'adep_height_6', 'sqrd_vlof_tas', 'average_airspeed_ARR_100', 'adep_height_7', 'wind_distance_ARR_100', 'altitude_4', 
        'adep_height_1', 'groundspeed_airspeed_ratio_ARR', 'tas_8', 'specific_energy_4', 'temperature_bins_DEP', 'temperature_6', 'humidity_bins_DEP', 
        'altitude_5', 'adep_height_5', 'sqrd_tas_8', 'sqrd_tas_7', 'specific_energy_7', 'specific_energy_1', 'adep_height_4', 'sqrd_tas_6', 'tas_2', 
        'sqrd_tas_5', 'specific_energy_3', 'altitude_8', 'specific_energy_6', 'adep_height_8', 'vertical_rate_airspeed_ARR', 'altitude_2', 'sqrd_tas_1', 
        'sqrd_tas_3', 'specific_energy_8', 'sqrd_tas_9', 'temperature_8', 'groundspeed_airspeed_ratio_DEP', 'sqrd_tas_4', 'altitude_6', 
        'specific_energy_5', 'humidity_temperature_DEP', 'adep_height_2', 'altitude_7', 'adep_height_3', 'temperature_1', 'specific_energy_2', 
        'temperature_5', 'wind_distance_flown_distance_ARR', 'arrival_month', 'temperature_4', 'groundspeed_ARR_100', 'tas_4', 'arrival_minute', 
        'adep_height_9', 'altitude_groundspeed_ARR', 'altitude_3', 'temperature_7', 'airspeed_specific_energy_ENR', 'altitude_10', 'sqrd_tas_10', 
        'humidity_bins_ARR', 'specific_energy_9', 'sqrd_tas_2', 'temperature_2', 'tas_10', 'average_humidity_ENR', 'offblock_quarter', 
        'airspeed_specific_energy_DEP', 'wind_distance_flown_distance_DEP', 'tas_6', 'flown_distance_ARR_100', 'vertical_rate_airspeed_ratio_ARR', 
        'average_humidity_ARR_100', 'specific_energy_10', 'first_adep_height', 'tas_3', 'temperature_3', 'track_variation_ARR_100', 
        'is_offblock_rush_hour', 'average_temperature_ENR', 'is_arrival_rush_hour', 'average_altitude_ARR_100', 'specific_energy_ENR', 
        'groundspeed_ENR', 'is_offblock_weekend', 'Num_Engines', 'temperature_bins_ARR', 'average_temperature_ARR_100', 'kpi17_time', 
        'average_airspeed_DEP_40', 'wind_distance_ENR', 'offblock_minute', 'groundspeed_10NM', 'average_vertical_rate_ARR_100', 'vlof_tas', 
        'humidity_temperature_ARR']

# eliminated_features = ['track_variation_ARR_100', 'vertical_rate_bins_ARR', 'adep_height_5', 'temperature_bins_DEP', 'is_offblock_weekend', 'humidity_temperature_DEP', 'humidity_bins_DEP', 'specific_energy_4', 'groundspeed_airspeed_ratio_ARR', 'average_vertical_rate_ARR_100', 'flown_distance_ARR_100', 'arrival_minute', 'max_altitude_ARR_100', 'wind_distance_flown_distance_ARR', 'adep_height_1', 'offblock_year', 'arrival_year', 'offblock_to_arrival_day_diff', 'adep_height_7', 'specific_energy_5', 'groundspeed_flown_distance_ARR', 'altitude_5', 'groundspeed_ARR_100', 'temperature_1', 'altitude_6', 'altitude_groundspeed_ARR', 'adep_height_3', 'altitude_7', 'temperature_bins_ARR', 'altitude_3', 'specific_energy_2', 'humidity_bins_ARR', 'specific_energy_1', 'altitude_8', 'specific_energy_3', 'temperature_3', 'sqrd_tas_6', 'adep_height_8', 'groundspeed_10NM', 'vertical_rate_airspeed_ARR', 'adep_height_2', 'offblock_quarter', 'temperature_7', 'wind_distance_ARR_100', 'average_humidity_DEP_40', 'adep_height_6', 'temperature_9', 'average_airspeed_ARR_100', 'adep_height_4', 'specific_energy_9', 'specific_energy_10', 'altitude_4', 'vlof_tas', 'specific_energy_ENR', 'sqrd_tas_9', 'temperature_2', 'altitude_10', 'arrival_quarter', 'average_humidity_ENR', 'altitude_1', 'sqrd_tas_4', 'temperature_10', 'specific_energy_7', 'wind_distance_flown_distance_ENR', 'tas_6', 'is_arrival_weekend', 'specific_energy_8', 'temperature_6', 'altitude_9', 'kpi17_time', 'average_humidity_ARR_100', 'tas_8', 'first_adep_height', 'tas_9', 'specific_energy_6', 'tas_1', 'vertical_rate_airspeed_ratio_ARR', 'average_altitude_ARR_100', 'temperature_5', 'adep_height_9', 'tas_7', 'avg_speed_ENR', 'groundspeed_ENR', 'altitude_groundspeed_DEP', 'tas_2', 'groundspeed_DEP_40']

In [13]:
# Dropping the eliminated features
X.drop(eliminated_features, axis=1, inplace=True)

In [14]:
from catboost import CatBoostRegressor, Pool, metrics
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and validation datasets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameters found using catboost-optuna.py
params = {
    'learning_rate': 0.01, 
    'reg_lambda': 0.05357182104973179, 
    'random_strength': 20.10951792232919, 
    'depth': 9, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 12,
}

# Since some categorical features may have been dropped, we need to remove them from the list of categorical features
selected_cat_names = [x for x in cat_names if x in X.columns]

train_pool = Pool(X_train, y_train, cat_features=selected_cat_names)
val_pool = Pool(X_val, y_val, cat_features=selected_cat_names)

model = CatBoostRegressor(
    iterations=100000,
    objective=metrics.RMSE(),
    eval_metric=metrics.RMSE(),
    random_seed=42,
    verbose=False,
    task_type='GPU',
    use_best_model=True,
    od_type='Iter',
    od_wait=50,
    **params,
)

In [15]:
# Training the model
model.fit(
    train_pool, eval_set=val_pool,
    verbose=100,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 52821.2848362	test: 52397.4845599	best: 52397.4845599 (0)	total: 173ms	remaining: 4h 48m 20s
100:	learn: 20409.5178707	test: 20258.2838954	best: 20258.2838954 (100)	total: 13.2s	remaining: 3h 36m 56s
200:	learn: 8829.2007729	test: 8783.0542872	best: 8783.0542872 (200)	total: 26.3s	remaining: 3h 37m 24s
300:	learn: 5217.7898084	test: 5215.6324718	best: 5215.6324718 (300)	total: 39.6s	remaining: 3h 38m 32s
400:	learn: 4223.8519331	test: 4238.5798512	best: 4238.5798512 (400)	total: 53.1s	remaining: 3h 39m 47s
500:	learn: 3882.9357228	test: 3906.3077407	best: 3906.3077407 (500)	total: 1m 6s	remaining: 3h 41m 14s
600:	learn: 3702.5101829	test: 3734.8060063	best: 3734.8060063 (600)	total: 1m 20s	remaining: 3h 42m 49s
700:	learn: 3575.7004039	test: 3613.9338275	best: 3613.9338275 (700)	total: 1m 34s	remaining: 3h 42m 45s
800:	learn: 3472.6790177	test: 3516.2095544	best: 3516.2095544 (800)	total: 1m 47s	remaining: 3h 42m 47s
900:	learn: 3385.2089925	test: 3433.9728767	best: 3433.9728

In [17]:
# Saving the model trained on the train set
model.save_model('catboost_train_v%d.cbm' % version, 'cbm')

In [18]:
# Saving the ATOWs estimated by the model for the flights on the val set (this is used to tune the ensemble model)
y_pred = model.predict(X_val)
pd.DataFrame(data={'tow': y_pred}).to_csv('catboost_val_v%d.csv' % version, index=False)

In [19]:
# Auxiliary function to categorize the aircraft
def classify_aircraft(row):
    if row['Physical_Class_Engine'] == 'Turboprop' and row['wtc'] == 'M':
        return 'Medium Turbo Prop'
    elif row['Physical_Class_Engine'] == 'Jet' and row['wtc'] == 'M':
        return 'Medium Jet'
    elif row['Physical_Class_Engine'] == 'Jet' and row['wtc'] == 'H':
        return 'Heavy Jet'
    else:
        return None  # If no classification applies, return None

In [20]:
# This computes separated RMSE for each airplane category. We found that most of our RMSE comes from the "Heavy Jet" aircraft.
from sklearn.metrics import root_mean_squared_error

X_val_groups = X_val.copy()

X_val_groups['Aircraft_Class'] = X_val.apply(classify_aircraft, axis=1)

X_val_groups = X_val_groups.reset_index()

indices_m_prop = X_val_groups[X_val_groups['Aircraft_Class'] == 'Medium Turbo Prop'].index
indices_m_jet = X_val_groups[X_val_groups['Aircraft_Class'] == 'Medium Jet'].index
indices_h_jet = X_val_groups[X_val_groups['Aircraft_Class'] == 'Heavy Jet'].index

print('RMSE Medium Turbo Prop:', root_mean_squared_error(y_val.iloc[indices_m_prop], y_pred[indices_m_prop]))
print('RMSE Medium Jet:', root_mean_squared_error(y_val.iloc[indices_m_jet], y_pred[indices_m_jet]))
print('RMSE Heavy Jet:', root_mean_squared_error(y_val.iloc[indices_h_jet], y_pred[indices_h_jet]))

RMSE Medium Turbo Prop: 754.9084396612558
RMSE Medium Jet: 1445.2313187083985
RMSE Heavy Jet: 4765.434966451966


In [21]:
# Retrain the model using the whole dataset for maximum performance on the challenge
best_iteration = model.get_best_iteration()
model = CatBoostRegressor(
    iterations=best_iteration,
    objective=metrics.RMSE(),
    random_seed=42,
    verbose=False,
    task_type='GPU',
    **params,
)
model.fit(
    X, y,
    cat_features=selected_cat_names, 
    plot=True,
    verbose=100
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 52737.2024037	total: 385ms	remaining: 6h 57m 33s
100:	learn: 20346.3274621	total: 15.4s	remaining: 2h 44m 40s
200:	learn: 8803.5004082	total: 31s	remaining: 2h 46m 52s
300:	learn: 5203.9076887	total: 47.1s	remaining: 2h 48m 57s
400:	learn: 4218.7403173	total: 1m 3s	remaining: 2h 51m 9s
500:	learn: 3886.0295635	total: 1m 19s	remaining: 2h 51m 10s
600:	learn: 3707.6974692	total: 1m 35s	remaining: 2h 50m 58s
700:	learn: 3584.4934152	total: 1m 51s	remaining: 2h 50m 22s
800:	learn: 3479.9575731	total: 2m 6s	remaining: 2h 49m 38s
900:	learn: 3391.2088629	total: 2m 22s	remaining: 2h 49m 30s
1000:	learn: 3311.9920549	total: 2m 38s	remaining: 2h 49m 18s
1100:	learn: 3242.4371271	total: 2m 53s	remaining: 2h 48m 28s
1200:	learn: 3180.3561533	total: 3m 9s	remaining: 2h 47m 54s
1300:	learn: 3119.5783009	total: 3m 24s	remaining: 2h 47m 28s
1400:	learn: 3061.5649239	total: 3m 40s	remaining: 2h 47m 13s
1500:	learn: 3007.4717259	total: 3m 56s	remaining: 2h 47m 15s
1600:	learn: 2955.5473723	to

In [22]:
# Saving the model trained on the whole dataset
model.save_model('catboost_all_v%d.cbm' % version, 'cbm')

In [23]:
# Getting feature importance
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

FAA_Weight: 15.770801621843264
seats_min: 10.435828535966493
airline: 7.846327930322203
oew_kg: 6.708402942969851
seats_max: 5.860259074465004
height_m: 5.488986978159058
length_m: 4.486220788284492
wtc: 4.305463233643743
Manufacturer: 3.9695208014889936
tas_knots: 3.1891775301257033
aircraft_type: 2.480341807405063
MTOW_kg: 2.1506861118109226
Approach_Speed_knot: 2.113402785696119
wingspan_m: 2.05447172189299
max_fuel_l: 1.6811437699462168
flight_duration_category: 1.2746116764432445
cargo_capacity: 1.2576833771225768
flown_distance: 1.2439605944861778
Model_FAA: 1.0804323869942447
seats_typ: 0.9593492883607175
MALW_kg: 0.9431039990273843
actual_distance: 0.9353513345083084
average_altitude_ENR: 0.8975022589120369
speed_per_distance: 0.8295361621452814
flown_distance_ENR: 0.6706492398683088
range_km: 0.596246477885841
mzfw_kg: 0.5907128258759952
flight_time_excl_taxi: 0.5618299838068418
ades: 0.4315105118381213
offblock_to_arrival_duration: 0.3690007198125077
cruise_altitude_ENR: 0.34

In [ ]:
# Plotting the feature importance

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

feature_im_df = pd.DataFrame({
    "feature": X.columns,
    "importance": feature_importances
})

feature_im_df = feature_im_df.sort_values(by="importance", ascending=False)

sns.set(rc={'text.usetex' : True, 'text.latex.preamble': '\\usepackage{libertine}'})
plt.figure(figsize=(10, 6))
plt.figure()
sns.barplot(data = feature_im_df[:20], x='importance', y='feature', palette="viridis")

plt.title("CatBoost feature importance")
plt.xlabel("Importance")
plt.ylabel("feature")
plt.tight_layout()
plt.savefig('catboost_feature_importance.pdf', bbox_inches='tight')
plt.show()

## Generating Submission

In [ ]:
# model.load_model('catboost_all_v%d.cbm' % version, 'cbm')

In [23]:
# Recover the submission set (after imputation)
df_test = dataset.iloc[challenge_set_updated.shape[0]:, :]
df_test.head()

,callsign,adep,ades,aircraft_type,wtc,airline,taxiout_time,flown_distance,track_variation_ARR_100,track_variation_DEP_40,...,Latitude_ades,Longitude_ades,Altitude_ades,actual_distance,altitude_difference,bearing,elevation_gradient,adep_geo_cluster,ades_geo_cluster,tow
0,3b3de0f3ad0ee192513995c02f7bf7cf,LTFJ,LFLL,B738,M,6351ec1b849adacc0cbb3b1313d8d39b,15.0,1122,1.668989,1.079187,...,45.726,5.091,251,2022.915548,-61,293.477205,-0.030154,11,17,63852.0
1,e06dd03d4a879ca37d9e18c1bd7cad16,EBBR,KJFK,A333,H,bdeeef3a675587d530de70a25d7118d2,15.0,3205,1.766098,1.147364,...,40.640,-73.779,4,5886.430370,-53,291.395141,-0.009004,6,1,63852.0
2,2d3b1c962c78c4ebeef11bcd51b9e94c,KMIA,EGLL,B77W,H,5543e4dc327359ffaf5b9c0e6faaf0e1,10.0,3965,6.253309,1.292737,...,51.477,-0.461,25,7108.920003,22,43.036806,0.003095,12,13,63852.0
3,35f7721f68bf85128195547ae38b0f04,EBBR,LEAL,B738,M,f53c55b5cf0cbb3be755bf50df6fa52d,9.0,802,1.775667,0.905718,...,38.282,-0.558,44,1458.405355,-13,197.753476,-0.008914,6,19,63852.0
4,eb56918bee9bc5204624186b9bcc4391,LSZH,LFPG,BCS3,M,2d5def0a5a844b343ba1b7cc9cb28fa9,11.0,292,1.200644,1.204058,...,49.013,2.550,120,476.291487,-312,293.398537,-0.655061,2,6,63852.0


In [25]:
X_test = df_test.drop('tow', axis=1)
X_test = X_test.loc[:, X_test.columns.isin(X.columns)]

In [26]:
# Predicting the estimated ATOWs
y_pred = model.predict(X_test)
y_test = y_pred
print(y_test)

[ 70519.07337769 215353.62737715 223748.37469714 ... 196580.87594207
  43118.03030744  63585.06153424]


In [27]:
# Consistency check to verify if we have the correct orders in the datasets (the output must be 1.0)
print((df_test['aircraft_type'] == submission_set_updated['aircraft_type']).mean())

1.0


In [28]:
# Heuristically limiting the ATOW to the maximum value for the given aircraft
submission_dataset = df_test
submission_dataset['tow'] = y_pred
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
print((submission_dataset['tow'] > submission_dataset['MTOW_kg']).sum())
mask = (submission_dataset['tow'] > submission_dataset['MTOW_kg'])
submission_dataset.loc[mask, 'tow'] = submission_dataset.loc[mask, 'MTOW_kg']
print((submission_dataset['tow'] > submission_dataset['MTOW_kg']).sum())

85
0


C:\Users\rudog\AppData\Local\Temp\ipykernel_32028\221719705.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_dataset['tow'] = y_pred


In [29]:
# Generating the submission file (after ATOW limitation)
dft0 = pd.read_csv('./data/final_submission_set.csv')
dft0['tow'] = submission_dataset['tow']
dft0[['flight_id', 'tow']].to_csv('catboost_submission_v%d.csv' % version, index=False)